In [2]:
import csv
from bs4 import BeautifulSoup  
import pandas as pd  
import requests
from pprint import pprint
from ast import literal_eval
import re
import time

Создаем список марок, интересующих нас, как они обозначены на сайте

In [ ]:
model_list = ['audi', 'bmw', 'honda', 'infiniti', 'lexus', 'mercedes', 'mitsubishi', 'nissan', 'skoda', 'toyota', 'volkswagen', 'volvo']

In [ ]:
def count_of_cars(url):
    
    '''функция считает количество объявлений по ссылке'''
    
    req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text, 'html.parser')
    count = soup.find('span', class_="ButtonWithLoader__content")
    pattern = re.compile('\d')
    number = ''
    for i in pattern.findall(count.text):
        number += i
    
    return int(number)

In [ ]:
def split_cars_on_parts(url_base):
    
    '''функция разделяет каждую марку по году производства, так что бы
    уложится в максимальное количество объявлений выводимых по запросу'''
    
    num_of_cars = count_of_cars(url_base)
    num_of_parts = int(num_of_cars / 3000) + 1
    url_list = []

    if num_of_cars <= 3670:
        print(url_base)
        url_list.append(url_base)
    else:
        year1 = 2016
        url = url_base + '?year_to=' + str(year1)
        if num_of_parts == 2:
            while count_of_cars(url) not in range(2900,3600):
                url = url_base + '?year_to=' + str(year1)
                year1 -= 1
                time.sleep(0.3)
            url2 = url_base + '?year_from=' + str(year1 + 2)
            print(url)
            url_list.append(url)
            print(url2)
            url_list.append(url2)
        elif num_of_parts > 2:
            while count_of_cars(url) not in range(2900,3600):
                year1 -= 1
                url = url_base + '?year_to=' + str(year1)
                time.sleep(0.3)
            print(url)
            url_list.append(url)
            year1 += 1
            year2 = year1 + 1
            for i in range(num_of_parts - 2):
                url = url_base + '?year_from=' + str(year1) + '&year_to=' + str(year2)
                while count_of_cars(url) not in range(2900,3600):
                    year2 += 1
                    url = url_base + '?year_from=' + str(year1) + '&year_to=' + str(year2)
                    time.sleep(0.3)
                print(url)
                url_list.append(url)
                year1, year2 = year2, year2 + 2
            print(url_base + '?year_from=' + str(year2))
            url_list.append(url_base + '?year_from=' + str(year2))
    
    return url_list

Адрес, на котором объявления о продаже авто.

In [ ]:
domen = 'https://auto.ru/cars/'

Создаем список адресов, по интересующим нас маркам, разделенные по году производства, так, что бы уложиться в максимальное число выводимых объявлений. По запросу максимум выдает 99 страниц, из них первые 7 по 38 объявлений, остальные  по 37, итого не более 3670 объявления.

In [ ]:
for model in model_list:
    url_base = domen + model + '/used/'
    req = requests.get(url_base, headers={'User-Agent': 'Mozilla/5.0'})
    req.encoding = 'utf-8'
    soup = BeautifulSoup(req.text, 'html.parser')
    car_url_list += split_cars_on_parts(url_base)
df = pd.DataFrame(car_url_list, columns=['url'])
df.to_csv('links.csv', index=False)

Сохраняем результат в файл

In [ ]:
urls = pd.read_csv('links.csv')

Проходим по каждой ссылке и сохраняем адрем каждого объявления в один список

In [ ]:
cars_list = []
df = pd.DataFrame(cars_list, columns=['url'])
for url in urls.url:
    num_of_cars = count_of_cars(url)
    num_of_sheets = (num_of_cars - 266) // 37 + 8
    for i in range(num_of_sheets):
        links = ''
        params = {'page': i+1}
        req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)
        soup = BeautifulSoup(req.text, 'html.parser')
        links = soup.find_all('a', class_="Link ListingItemTitle__link")
        cars = len(links)
        #time.sleep(0.4)
        for link in range(cars):
            cars_list.append(links[link]['href'])
            print(links[link]['href'])
            print(len(cars_list))
            df = pd.DataFrame(cars_list, columns=['url'])
            df.to_csv('cars_Total.csv', index=False)

Проходим по всему списку объявлений и копируем всю интересующую нас техническую информацию и каждую строчку записываем в файл, так как занимает очень много времени и возможны различные сбои, а так же меньше загружает память.

In [ ]:
cars = open('total_car_info.csv', 'w', encoding="utf-8")
csv_writer = csv.writer(cars)
csv_writer.writerow(['car_url', 'info1', 'info2', 
                     'info2owner', 'info2Card', 'info3', 
                     'info4', 'comp_link', 'time'])

In [ ]:
links = pd.read_csv('cars_Total.csv')

In [ ]:
for url in links.url:
    try:
        req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        req.encoding = 'utf-8'
        soup = BeautifulSoup(req.text, 'html.parser')
        dff1 = pd.DataFrame(index=[0], columns=['car_url', 'info1', 'info2', 'info2owner', 'info2Card', 'info3', 'info4', 'comp_link', 'time'])
        car_url = url
        info_1 = soup.find_all('script', type="application/ld+json")
        info1 = literal_eval(str(info_1[0]).strip('<script type="application/ld+json"> </script>'))
        info_2 = soup.find_all('div', class_="CardOfferBody__leftColumn")
        info2 = soup.find_all('div', class_="CardOfferBody__leftColumn")
        info2owner = info_2[0].find_all('div', class_="CardBenefits__item-title")
        info2Card = info_2[0].find_all('span', class_="CardInfoRow__cell")
        info3 = soup.find_all('section', class_="CardComplectation CardOfferBody__contentIsland")
        info_4 = soup.find_all('div', id="sale-data-attributes")
        info4 = literal_eval(str(info_4[0]).strip('<div data-bem=' 'id="sale-data-attributes" style="display:none"></div>'))
        comp_link_1 = soup.find_all('a', class_="Link SpoilerLink CardCatalogLink SpoilerLink_type_default")
        comp_link = comp_link_1[0]['href']
        time = datetime.today()
        print(links[links['url'] == url].index)
        csv_writer.writerow(
                [car_url, info1, info2, 
                     info2owner, info2Card, info3, 
                     info4, comp_link, time])
    except Exception as e:
            print(e)

По каждому объявлению мы сохранили ссылку на описание комплектации, они повторяются. Создадим список уникальных ссылок и скачаем информацию из них. А так как в тестовых данных нет этой ссылки, добавим общую информацию по каждому такому авто, что бы потом найти соотвествие для тестовых данных.

In [6]:
dataset = pd.read_csv('total_car_info.csv')

C:\Users\negat\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
data_complect = pd.DataFrame()
columns = ['comp_link', 'engineDisplacement', 'enginePower', 'model_name', 'vehicleConfiguration', 'vehicleTransmission']
data_complect['comp_link'] = dataset.comp_link.unique()


In [ ]:
data_complect['engineDisplacement'] = None
data_complect['enginePower'] = None
data_complect['model_name'] = None
data_complect['vehicleConfiguration'] = None
data_complect['vehicleTransmission'] = None
for i in range(len(data_complect)):
    data_complect['engineDisplacement'][i] = dataset.engineDisplacement[dataset.comp_link == data_complect.comp_link[i]].iloc[0]
    data_complect['enginePower'][i] = dataset.enginePower[dataset.comp_link == data_complect.comp_link[i]].iloc[0]
    data_complect['model_name'][i] = dataset.model_name[dataset.comp_link == data_complect.comp_link[i]].iloc[0]
    data_complect['vehicleConfiguration'][i] = dataset.vehicleConfiguration[dataset.comp_link == data_complect.comp_link[i]].iloc[0]
    data_complect['vehicleTransmission'][i] = dataset.vehicleTransmission[dataset.comp_link == data_complect.comp_link[i]].iloc[0]

In [ ]:
data_complect.to_csv('comp_links.csv', index=False)

In [4]:
data = pd.read_csv('comp_links.csv')

Скачиваем информацию по комплектациям и записываем в файл.

In [ ]:
data['comp_dict'] = ''

In [ ]:
for i in range(len(data)):
    try:
        req2 = requests.get(data.comp_link[i], headers={'User-Agent': 'Mozilla/5.0'})
        req2.encoding = 'utf-8'
        soup2 = BeautifulSoup(req2.text, 'html.parser')

        dd = soup2.find_all('dd', class_="list-values__value")

        dt = soup2.find_all('dt', class_="list-values__label")

        dic = {}


        for (part, name) in zip(dt, dd):
            dic.setdefault(part.text, name.text)
        data['comp_dict'][i] = dic
        print(i)
    except Exception as e:
        print(e)

In [ ]:
data.to_csv('comp_links_dicts.csv')